In [163]:
import pandas as pd
import numpy as np

In [164]:
df = pd.read_csv('tidy_2.csv')

## 새로운 칼럼 만들기

### 1) 대상인원 칼럼 만들기

In [165]:
df['대상인원'] = df['대상인원'].copy().str.extract(r'[\s|외|총|등](\d+)\s?[명|인]?')
# 대상인원이 없는 경우 운전기사 회식, 격려금 지급 등 맛집과 상관없는 경우가 많으니 삭제
df = df.dropna(axis=0, subset=['대상인원'], how='all')
df['대상인원'] = df['대상인원'].copy().astype(int)

### 2) 집행지명으로 그룹화하기

In [166]:
df_group = df.groupby('집행지명')\
            ['집행지명', 'cnt', '집행금액', '대상인원']\
            .sum().reset_index()
df_group.head(3)

,집행지명,cnt,집행금액,대상인원
0,(월)월향,1,34000.0,2
1,(주)강가,7,857000.0,30
2,(주)광화문아띠,1,40000.0,2


In [167]:
# 회당, 인당 금액을 구해보자. 
df_group['회당집행금액'] = df_group['집행금액'] / df_group['cnt']
df_group['인당집행금액'] = df_group['집행금액'] / df_group['대상인원']
df_group['회당명수'] = df_group['대상인원'] / df_group['cnt']
df_group.head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
0,(월)월향,1,34000.0,2,34000.000000,17000.000000,2.000000
1,(주)강가,7,857000.0,30,122428.571429,28566.666667,4.285714
2,(주)광화문아띠,1,40000.0,2,40000.000000,20000.000000,2.000000


## 카테고리별 맛집

### 1. 가장 자주 간 식당 (전체 공무원)

In [168]:
df_group.sort_values(by='cnt', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620.0,4653,116834.239766,17174.859231,6.802632
1006,동해일식,377,55124400.0,2248,146218.567639,24521.530249,5.962865
1462,바닷가작은부엌,310,45153200.0,1920,145655.483871,23517.291667,6.193548


### 2. 가장 돈을 많이 쓴 식당 (전체 공무원)

In [169]:
df_group.sort_values(by='집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620.0,4653,116834.239766,17174.859231,6.802632
1006,동해일식,377,55124400.0,2248,146218.567639,24521.530249,5.962865
1103,라칸티나,271,48128400.0,1812,177595.571956,26560.927152,6.686347


### 3. 회당 집행금액이 가장 많은 식당
(10회 이상 방문, 전체 공무원)

In [171]:
df_group_5_up = df_group[df_group['cnt']>50]
df_group_5_up.sort_values(by='회당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
868,더미,107,32091000.0,1227,299915.887850,26154.034230,11.467290
2913,진미족발,55,14689500.0,762,267081.818182,19277.559055,13.854545
2004,송학,60,15229000.0,583,253816.666667,26121.783877,9.716667


### 4. 인당 집행금액이 가장 많은 식당
(10회 이상 방문, 전체 공무원)

In [172]:
df_group_5_up = df_group[df_group['cnt']>50]
df_group_5_up.sort_values(by='인당집행금액', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
1372,몽중헌,62,12552500.0,403,202459.677419,31147.642680,6.500000
1756,삼원,63,13119000.0,478,208238.095238,27445.606695,7.587302
1291,메이징에이,107,17491300.0,649,163470.093458,26951.155624,6.065421


In [173]:
df_group_5_up.sort_values(by='인당집행금액', ascending=False).tail(10)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2590,육대장,93,5491700.0,434,59050.537634,12653.686636,4.666667
2754,장안삼계탕,68,9389001.0,751,138073.544118,12501.998668,11.044118
2576,유림분식집,63,2624400.0,224,41657.142857,11716.071429,3.555556
1949,소라,78,3908600.0,339,50110.256410,11529.793510,4.346154
2470,완산골명가,56,2791500.0,251,49848.214286,11121.513944,4.482143
738,농민백암순대,57,2763400.0,252,48480.701754,10965.873016,4.421053
1541,버거킹,58,2789400.0,325,48093.103448,8582.769231,5.603448
3285,폴바셋,67,1167800.0,174,17429.850746,6711.494253,2.597015
774,다락,113,3420900.0,1084,30273.451327,3155.811808,9.592920
2923,진주회관,115,6459000.0,2537,56165.217391,2545.920378,22.060870


### 5. 가장 많은 사람이 간 식당

In [175]:
df_group_5_up = df_group[df_group['cnt']>50]
df_group_5_up.sort_values(by='대상인원', ascending=False).head(3)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2966,참숯골,684,79914620.0,4653,116834.239766,17174.859231,6.802632
2923,진주회관,115,6459000.0,2537,56165.217391,2545.920378,22.060870
1006,동해일식,377,55124400.0,2248,146218.567639,24521.530249,5.962865


### 6. 단체손님이 많이 가는 식당

In [176]:
df_group_for_many = df_group_5_up[df_group_5_up['회당명수']>10]
df_group_for_many.sort_values(by='cnt', ascending=False).head(6)

,집행지명,cnt,집행금액,대상인원,회당집행금액,인당집행금액,회당명수
2343,열빈,191,38018500.0,1943,199049.738220,19566.906845,10.172775
1840,서울삼계탕,144,19542000.0,1518,135708.333333,12873.517787,10.541667
2923,진주회관,115,6459000.0,2537,56165.217391,2545.920378,22.060870
868,더미,107,32091000.0,1227,299915.887850,26154.034230,11.467290
2754,장안삼계탕,68,9389001.0,751,138073.544118,12501.998668,11.044118
2913,진미족발,55,14689500.0,762,267081.818182,19277.559055,13.854545


## 부서별로 살펴보기

In [177]:
# 부서별로 묶기
df_divs = df.groupby('1소속부서')['cnt', '대상인원', '집행금액']\
        .sum().reset_index()
df_divs.head(3)

,1소속부서,cnt,대상인원,집행금액
0,감사위원회,662,3872,80537330.0
1,경제진흥본부,141,931,15037300.0
2,관광체육국,1088,6450,104698248.0


In [178]:
# 인당/회당 금액 내기
df_divs['인당집행금액'] = df_divs['집행금액'] / df_divs['대상인원']
df_divs['회당집행금액'] = df_divs['집행금액'] / df_divs['cnt']

### 1. 가장 회식을 자주 한 부서

In [179]:
df_divs.sort_values(by='cnt', ascending=False).head(3)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
25,행정국,2388,14993,297689495.0,19855.232108,124660.592546
16,시민소통기획관,2045,13525,276098922.0,20413.968355,135011.697800
20,정무부시장,1534,8996,228080150.0,25353.507114,148683.279009


### 2. 가장 돈을 많이 쓴 부서

In [180]:
df_divs.sort_values(by='집행금액', ascending=False).head(3)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
25,행정국,2388,14993,297689495.0,19855.232108,124660.592546
16,시민소통기획관,2045,13525,276098922.0,20413.968355,135011.697800
24,행정2부시장,722,12019,230972660.0,19217.294284,319906.731302


관광체육국은 자주 회식을 하는 것에 비해 돈을 많이 쓰지는 않으니 싸고 맛있는 집을 많이 가겠군! 라고 추론해보았다. 

### 3. 가장 비싼 것을 먹는 부서

In [181]:
df_divs.sort_values(by='인당집행금액', ascending=False).head(5)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
23,행정1부시장,721,6569,166838070.0,25397.788096,231398.155340
20,정무부시장,1534,8996,228080150.0,25353.507114,148683.279009
11,비상기획관,359,1945,42716760.0,21962.344473,118988.189415
6,대변인,1437,7721,161784939.0,20953.884082,112585.204593
0,감사위원회,662,3872,80537330.0,20799.930269,121657.598187


### 4. 가장 싼 것을 먹는 부서

In [182]:
df_divs.sort_values(by='인당집행금액', ascending=True).head(5)

,1소속부서,cnt,대상인원,집행금액,인당집행금액,회당집행금액
5,기후환경본부,1252,8921,114193360.0,12800.511153,91208.753994
3,기술심사담당관,114,1030,14562200.0,14138.058252,127738.596491
7,도시공간개선단,318,1746,26354982.0,15094.491409,82877.301887
13,서울혁신기획관,1296,7024,108600810.0,15461.390945,83796.921296
19,재무국,1313,8837,138009766.0,15617.264456,105110.255903


행정1부시장에 비해 행정2부시장이 가성비맛집을 많이 갈 확률이 높다!